In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

# Experimental distance-frequency matrix

## Old experiments

In [ ]:
from evaluate_data import read_df, integrate_yaw
from dynamic_analysis import add_pose_to_df
from wall_analysis import get_psd

exp_name = '2020_11_23_wall2'; DISTANCE_LIST = np.arange(100, step=10)
#exp_name = '2020_11_26_wall'; DISTANCE_LIST = np.arange(50)

#fname = f'results/{exp_name}_simulated.pkl'
fname = f'results/{exp_name}_real.pkl'

SOURCE_LIST = ['mono4125', 'mono3500', None, 'sweep', 'sweep_low', 'sweep_high'] # 
DEGREE_LIST = [0, 27, 54, 81, 360]

try:
    df_total = pd.read_pickle(fname)
    frequencies = df_total.iloc[0].frequencies
    print('read', fname)
except:
    print('could not read', fname)
    df_total = pd.DataFrame(columns=['signals_f', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 
                                     'frequencies'])

    params = dict(
      props = False,
      snr = False,
      motors = False,
      exp_name = exp_name
    )
    
    for degree in DEGREE_LIST:
        for distance in DISTANCE_LIST:
            for source in SOURCE_LIST:
                try:
                    params['degree'] = degree
                    params['distance'] = distance
                    params['source'] = source
                    params['appendix'] = ""
                    if (exp_name == '2020_11_23_wall2') and (distance in [10, 30, 50]):
                        params['appendix'] = "_new"
                    df, df_pos = read_df(**params)
                except Exception as e:
                    continue 

                # detect index decrease (happens when two csv files are concatenated)
                sign = np.sign(df['index'].values[1:] - df['index'].values[:-1])
                if np.any(sign < 0):
                    index = np.where(sign<0)[0][-1]
                    print('Warning: found multiple start indices, start at', index)
                    df = df.iloc[index:]
                    index_start = df.iloc[0]['index']
                    df_pos = df_pos.loc[df_pos.index >= index_start]

                signals_f = np.array([*df.signals_f.values]) # n_times x n_mics x n_freqs
                frequencies_matrix = np.array([*df.loc[:,'frequencies']])
                frequencies = frequencies_matrix[0, :]
                assert not np.any(np.any(frequencies_matrix - frequencies[None, :], axis=0))

                if degree == 360:
                    add_pose_to_df(df, df_pos, max_allowed_lag_ms=50)
                    yaw = integrate_yaw(df.timestamp.values, df.yaw_rate_deg.values)
                else:
                    yaw = np.full(len(df), -degree)

                spec = np.sum(np.abs(signals_f), axis=1)
                psd = get_psd(signals_f, frequencies, fname='real')

                df_total.loc[len(df_total), :] = dict(
                    degree=degree,
                    yaw=yaw,
                    distance=distance,
                    source=str(source),
                    signals_f=signals_f,
                    frequencies=frequencies,
                    spec=spec,
                    psd=psd
                )

    fname = f'results/{exp_name}_real.pkl'
    pd.to_pickle(df_total, fname)
    print('saved as', fname)

## New experiments

In [ ]:
#exp_name = '2020_12_7_moving'; 
exp_name = '2020_12_9_rotating'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    print('Error: run wall_analysis.py to parse experiments.')

In [ ]:
df_total.tail()

## sanity checks

In [ ]:
row_wav = df_total.loc[df_total.mic_type=='measurement', :].iloc[0]
fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
ax.pcolorfast(row_wav.seconds, row_wav.frequencies, np.log(row_wav.spec.T))

In [ ]:
for i, row in df_total.iterrows():
    plt.plot(row.seconds, row.seconds + i * 10, label=f'row {i}')
#plt.legend(bbox_to_anchor=(1.0, 0), loc='lower left')

In [ ]:
df_meas = df_total.loc[df_total.mic_type == 'audio_deck', :]
for chosen_source, df_this in df_meas.groupby('source'):
    for tuple_params, df_app in df_this.groupby(['appendix', 'snr', 'distance', 'degree']):
        
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(15, 5)
        fig.suptitle(f'{chosen_source}, {str(tuple_params)}')

        for j, (motors, df_motors) in enumerate(df_app.groupby('motors')):
            assert len(df_motors) <= 1, f"{len(df_motors)} findings for {tuple_params, motors, chosen_source}: {df_motors}"
            
            row = df_motors.iloc[0]
            frequencies = row.frequencies
            
            signals_f = row.signals_f
            if motors != 'all43000':
                signals_f = signals_f[::-1, :, :]
            
            if not chosen_source in ('None', None):
                try:
                    freq = int(chosen_source.strip('mono'))
                    bin_ = np.argmin(abs(frequencies - freq))
                    assert int(frequencies[bin_]) == freq
                except: 
                    bin_ = 15
            else: 
                bin_ = np.argmax(np.sum(np.abs(signals_f), axis=(0, 1))) 
                print('max frequency without source:', frequencies[bin_])
                
            seconds = row.seconds

            for mic_idx in range(4):
                spec = np.abs(signals_f[:, mic_idx, :]) # times x frequencies
                axs[j].plot(seconds, spec[:, bin_], label=f'mic{mic_idx}')
                axs[j].set_title(f'motors {motors}')
            
            axs[j].legend(loc='upper right')
            #axs[j].set_ylim(0, 2)
            axs[j].set_xlim(0, 180)

In [ ]:
from wall_analysis import filter_by_dicts
if exp_name == "2020_12_7_moving":
    motors = "all43000"
    chosen_dicts = [
        {"source": "mono3125", "motors": 0, "appendix":""},
        {"source": "mono3125", "motors": motors, "appendix":"_new"},
        {"source": "mono4156", "motors": 0, "appendix":""},
        {"source": "mono4156", "motors": motors, "appendix":"_new"},
        {"source": "mono8000", "motors": 0, "appendix":"_new"},
        {"source": "mono8000", "motors": motors, "appendix":""},
        {"source": "None", "motors": motors, "appendix":"_new"},
    ]
elif exp_name == "2020_12_9_rotating":
    snr = 0
    appendix = ""
    chosen_dicts = []
    for d in df_total.distance.unique():
        for deg in df_total.degree.unique():
            if deg == 0:
                motors = ["all43000", 0]
            else:
                motors = [0]
                
            for m in motors:
                chosen_dicts += [
                    {"motors":m, "appendix":appendix, "snr":snr, "distance":d, "degree":deg}
                ]
    
df_chosen = filter_by_dicts(df_total, chosen_dicts)
assert len(df_chosen) == 2*len(chosen_dicts), len(df_chosen)# times 2 because of mic_type
df_chosen.tail()

## spectrograms

In [ ]:
mic_type = 'measurement'
cut_x = 500
cut_y = range(100, 400)

filters = ['source', 'degree', 'distance']
for chosen_tuple, df_source in df_chosen[df_chosen.mic_type==mic_type].groupby(filters):
    fig, axs = plt.subplots(1, 3, squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(dict(zip(filters, chosen_tuple)))

    for j, motors in enumerate([0, "all43000"]):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            print(len(df_this))
            continue

        row = df_this.iloc[0]
        axs[0, j].pcolorfast(row.seconds, row.frequencies, np.log(row.spec.T))
        axs[0, j].set_title(f'motors {motors}')

    axs[0, 1].plot([row.seconds[0], row.seconds[cut_x]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[0]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[0], row.seconds[0]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[cut_x], row.seconds[cut_x]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[0], row.seconds[cut_x]], [row.frequencies[cut_y[-1]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    axs[0, 2].pcolorfast(row.seconds[:cut_x], row.frequencies[cut_y], np.log(row.spec.T[cut_y, :cut_x]))
    fig.savefig(f'/home/duembgen/Desktop/spec-{chosen_source}.png', bbox_inches='tight')

## frequency selection for propeller noise

In [ ]:
df_source = df_chosen.loc[df_chosen.source=="None"]
df_mic = df_source.loc[df_source.mic_type == "audio_deck"]
assert len(df_mic) == 1
row = df_mic.iloc[0]
f_matrix = row.frequencies_matrix
print(f_matrix.shape)
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
for i, row in enumerate(f_matrix):
    plt.scatter([i]*len(row), row, color='C0', s=1)

In [ ]:
#levels = list(range(2))
levels = list(range(5))
num_levels = len(levels)
#times = range(f_matrix.shape[0]) # range(100, 300)
times = range(10, 800)
f_matrix_reduced = f_matrix[times, :]
f_matrix_reduced = f_matrix_reduced[:, levels]
fig, axs = plt.subplots(num_levels, sharey=False, sharex=True, squeeze=False)
fig.set_size_inches(10, 10)
for j, level in enumerate(levels):
    axs[j, 0].plot(times, f_matrix_reduced[:, j], color=f"C{j}", marker='o')
    freq = np.median(f_matrix_reduced[:, j])
    axs[j, 0].set_title(f'level {level}: {freq}Hz')

In [ ]:
all_freqs = np.sort(f_matrix[times,:10].flatten())
count, bins = np.histogram(all_freqs, bins=np.unique(all_freqs))
plt.plot(bins[:-1], count)
plt.title('histogram strongest 10 frequency bins')
print(bins[np.argsort(count)[::-1]])

In [ ]:
all_freqs = np.sort(f_matrix[times,0].flatten())
count, bins = np.histogram(all_freqs, bins=np.unique(all_freqs))
plt.plot(bins[:-1], count)
plt.title('histogram strongest frequency bin')
print(bins[np.argsort(count)[::-1]])

## distance-slices

In [ ]:
if exp_name != '2020_12_7_moving':
    raise ValueError('cannot do distance slices on this dataset')
#mic_indices = range(4)
from constants import SPEED_OF_SOUND
CHOSEN_MICS = [0, 1, 2, 3] #[3]
CUT_LAST = 3
CUT_FIRST = 3
X_LIM = 20
#CHOSEN_PROP_FREQ = 781 #
#CHOSEN_PROP_FREQ = 671 #
#CHOSEN_PROP_FREQ = 734 #
#CHOSEN_PROP_FREQ = 2187 #1
CHOSEN_PROP_FREQ = 1453 

for chosen_source, df_source in df_chosen.groupby('source'):
    for mic_type, df_this in df_source.groupby('mic_type'):
            
        if mic_type == 'audio_deck':
            mic_indices = CHOSEN_MICS
        else:
            mic_indices = [0]
            
        fig, axs = plt.subplots(2, 2, squeeze=False)
        fig.set_size_inches(15, 5)
        fig.suptitle(chosen_source)

        for j, motors in enumerate([0, 'all43000']):
            
            df_motors = df_this.loc[df_this.motors==motors]
            
            if len(df_motors) != 1: 
                print(f"{len(df_motors)} findings for {motors, chosen_source}")
                continue

            row = df_motors.iloc[0]

            signals_f = row.signals_f
            if motors != 'all43000':
                signals_f = signals_f[::-1, :, :]

            if (chosen_source == "None") and (mic_type == "audio_deck"):
                freq = CHOSEN_PROP_FREQ
                bin_ = np.argmin(np.abs(row.frequencies_matrix - freq), axis=1)
            elif chosen_source != "None":
                freq = int(chosen_source.strip('mono'))
                bin_ = np.argmin(abs(row.frequencies - freq))
                error = abs(row.frequencies[bin_] - freq)
                if error > 10:
                    print(f'Warning: big diff between {freq} and {row.frequencies[bin_]}')
            else:
                freq = CHOSEN_PROP_FREQ
                bin_ = np.argmin(abs(row.frequencies - freq))
                error = abs(row.frequencies[bin_] - freq)
                if error > 10:
                    print(f'Warning: big diff between {freq} and {row.frequencies[bin_]}')

            seconds = row.seconds[CUT_FIRST:-CUT_LAST]
            distances = seconds * 50 / 165.0
            mean_d = np.median(distances[5:]-distances[4:-1])
            print(mean_d)
            expected_period = SPEED_OF_SOUND / freq * 100 / 2 #

            for mic_idx in mic_indices:
                spec = np.abs(signals_f[range(signals_f.shape[0]), mic_idx, bin_])[CUT_FIRST:-CUT_LAST] # times x frequencies
                axs[0, j].plot(distances, spec, label=f'mic{mic_idx}', color=f'C{mic_idx}')
                axs[0, j].set_title(f'motors {motors}')
                
                spec_fft = np.fft.rfft(spec)[1:]
                freqs = np.fft.rfftfreq(len(spec), mean_d)[1:]
                period = 1/freqs
                axs[1, j].semilogx(period, np.abs(spec_fft), color=f'C{mic_idx}')
                axs[1, j].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm

            axs[0, j].legend(loc='upper right')
            axs[0, j].set_xlim(0,50)
            axs[1, j].set_xlim(1e-1,50)
            #axs[0, j].set_ylim(min(spec), max(spec))

## frequency slices

In [ ]:
if exp_name != '2020_12_9_rotating':
    raise ValueError('cannot do frequency slices')
    
from bin_selection import select_frequencies
from constants import SPEED_OF_SOUND
    
sys.path.append(f'../experiments/{exp_name}/')
from params import MIN_FREQ, MAX_FREQ
    
CHOSEN_MICS = [0, 1, 2, 3] #[3]

for (distance,mic_type,motors), df_this in df_chosen.groupby(['distance', 'mic_type', 'motors']):
    if mic_type == 'audio_deck':
        mic_indices = CHOSEN_MICS
        bins_ = range(32)
    else:
        mic_indices = [0]
        bins_ = select_frequencies(N_BUFFER, 44100, min_freq=MIN_FREQ, max_freq=MAX_FREQ)

    fig, axs = plt.subplots(2, len(mic_indices), squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}')

    if len(df_this) != 1: 
        print(f"{len(df_this)} findings for {motors, chosen_source}")
        continue

    row = df_this.iloc[0]
    signals_f = row.signals_f # times x n_mics x frequencies
    frequencies = row.frequencies
    
    mean_f = frequencies[bins_[1]] - frequencies[bins_[0]] 
    expected_period = SPEED_OF_SOUND / (2 * distance * 1e-2) # m/s / m = Hz

    for mic_idx in mic_indices:
        spec = np.sum(np.abs(signals_f[:, mic_idx, bins_]), axis=0) # times x frequencies
        axs[0, mic_idx].plot(frequencies[bins_], spec, label=f'mic{mic_idx}', color=f'C{mic_idx}')
        axs[0, mic_idx].set_title(f'm{mic_idx}')

        spec_fft = np.fft.rfft(spec)[1:]
        freqs = np.fft.rfftfreq(len(spec), mean_f)[1:] # "seconds"
        axs[1, mic_idx].plot(1/freqs, np.abs(spec_fft), color=f'C{mic_idx}')
        axs[1, mic_idx].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm

    #axs[0, mic_idx].legend(loc='upper right')
    #axs[0, mic_idx].set_xlim(0,50)
    #axs[1, mic_idx].set_xlim(1e-1,50)
    #axs[0, j].set_ylim(min(spec), max(spec))

## Old analysis

In [ ]:
#for deg in [0, 27, 54, 81]:
    #df_total.loc[df_total.degree==deg, 'source'] = 'sweep'

In [ ]:
n_mics = df_total.iloc[0].signals_f.shape[1]
df_matrix_mics = np.empty((n_mics, len(frequencies), len(distances)))
normalize = False

if normalize:
    psd_ref = df_total[df_total.distance==distances[-1]].iloc[0].psd

for j, distance_cm in enumerate(distances):
    row = df_total[df_total.distance==distance_cm].iloc[0]
    for i in range(n_mics):
        if normalize:
            df_matrix_mics[i, :, j] = row.psd[i] / psd_ref[i]
        else:
            df_matrix_mics[i, :, j] = row.psd[i]

In [ ]:
fig, axs = plt.subplots(1, n_mics)
fig.set_size_inches(10, 5)
for i in range(n_mics):
    axs[i].pcolormesh(distances, frequencies, df_matrix_mics[i])

In [ ]:
from constants import SPEED_OF_SOUND
mic_idx = 0
for f_idx, slice_f in enumerate(np.arange(32)):
    
    fig, axs = plt.subplots(2, n_mics)
    fig.set_size_inches(15, 5)
    
    f = frequencies[slice_f]
    expected_period = SPEED_OF_SOUND / f * 1e2 / 2
    
    for mic_idx in range(n_mics):
        distance_response = df_matrix_mics[mic_idx, slice_f, :]
        axs[0, mic_idx].semilogy(distances, distance_response, label=f'{f:.0f}Hz')
        axs[0, mic_idx].set_xlabel('distance [cm]')
        axs[0, mic_idx].set_title(f'mic{mic_idx}')
        axs[0, mic_idx].legend(loc='lower left')

        distance_fft = np.fft.rfft(distance_response)[1:]
        distance_freq = 1 / np.fft.rfftfreq(len(distances), 1)[1:] # cm 
        axs[1, mic_idx].loglog(distance_freq, np.abs(distance_fft))
        axs[1, mic_idx].axvline(expected_period, ls=":")
        axs[1, mic_idx].set_xlabel('period [cm]')
    
    axs[0, 0].set_ylabel('PSD')

# Fixed angle analysis

In [ ]:
try:
    row = df_total[(df_total.distance==40) & (df_total.degree==0) & (df_total.source=='sweep')].iloc[0]
    spec = np.sum(np.abs(row.signals_f), axis=1)

    plt.figure()
    times = np.arange(spec.shape[0])
    plt.pcolormesh(times, frequencies, np.log10(spec.T))

    psd = get_psd(row.signals_f, frequencies, ax=plt.gca(), fname='real')

    plt.figure()
    for i_mic in range(psd.shape[0]):
        plt.semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD')
    plt.title(f'degree {degree}, distance {distance}')
except ValueError:
    raise
except:
    print('did not find 90, 81, sweep in')
    print(df_total.distance.unique())
    print(df_total.degree.unique())
    print(df_total.source.unique())

In [ ]:
degrees = [d for d in df_total.degree.unique() if d != 360][:2]
distances = df_total.distance.unique()[:3]
source = 'sweep'

fig, axs = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True)
fig.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])

fig_psd, axs_psd = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True)
fig_psd.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])
for i, distance in enumerate(distances):
    for j, degree in enumerate(degrees):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        row = df_this.iloc[0]
        spec = row.spec
        psd = row.psd
        if psd is None:
            psd = get_psd(row.signals_f)
        
        axs[i, j].pcolormesh(range(spec.shape[0]), frequencies, np.log10(spec.T))
        for i_mic in range(psd.shape[0]):
            axs_psd[i, j].semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
        axs[0, j].set_title(f'{degree} deg')
        axs_psd[0, j].set_title(f'{degree} deg')
    axs[i, 0].set_ylabel(f'{distance} cm')
    axs_psd[i, 0].set_ylabel(f'{distance} cm')
    
[axs[-1, j].set_xlabel(f'time idx') for j in range(len(degrees))]
[axs_psd[-1, j].set_xlabel(f'frequency [Hz]') for j in range(len(degrees))]

In [ ]:
from constants import SPEED_OF_SOUND

distance_ref = 90
distances = df_total.distance.unique()[:3]
source = 'sweep'
n_mics = df_ref.iloc[0].signals_f.shape[1]
print('n_mics:', n_mics)
    
for degree in [0, 27, 54, 81][:2]:
    
    df_ref = df_total.loc[(df_total.distance == distance_ref)
                           & (df_total.degree == degree)
                           & (df_total.source == source)]

    fig, axs = plt.subplots(1, n_mics, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    
    for mic in range(n_mics):
        psd_ref = df_ref.iloc[0].psd[mic] 
        for i, distance in enumerate(distances):
            df_this = df_total.loc[(df_total.distance == distance)
                                   & (df_total.degree == degree)
                                   & (df_total.source == source)]
            axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic], label=distance, color=f"C{i}")
            #axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic] / psd_ref, label=distance, color=f"C{i}")
            #axs[mic].plot(frequencies, df_this.iloc[0].psd[mic] - psd_ref, label=distance, color=f"C{i}")

        #axs[mic].set_xlim(min(frequencies), max(frequencies))
        axs[mic].set_xlim(2000,  max(frequencies))
        axs[mic].set_title(f"mic{mic}")
    
    fig.suptitle(degree)
    axs[mic].legend()

In [ ]:
#chosen_frequencies = frequencies[[10, 20, 30]]
chosen_frequencies = frequencies[[30]]
print(chosen_frequencies)

distance_ref = 90
mic = 0

for degree in [0, 27, 54, 81][:2]:
    distances = df_total.distance.unique()
    source = 'sweep'

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 5)
    for i, distance in enumerate(distances):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        row = df_this.iloc[0]
        for f, freq in enumerate(chosen_frequencies):
            chosen_idx = np.where(frequencies == freq)[0][0]
            ax.scatter(distance, row.psd[mic, chosen_idx], color=f"C{f}")
    ax.set_title(degree)
    #ax.set_yscale('log')
    ax.legend(chosen_frequencies)
    ax.set_xlabel('distance [cm]')

# Moving analysis

In [ ]:
#freq = 4125
#source = 'None'
source = 'mono4125'
degree = 360

distances = df_total.distance.unique()[:3]
chosen_idx = np.where(frequencies == freq)[0][0]

averages = []
for i, distance in enumerate(distances):
    fig, axs = plt.subplots(2)
    fig.set_size_inches(15, 5)
    
    ax = axs[0]
    df_this = df_total.loc[(df_total.distance == distance)
                           & (df_total.degree == degree)]
    df_this = df_this.loc[df_this.source == source]
                           
    signals_f = df_this.iloc[0].signals_f
    yaw =  df_this.iloc[0].yaw
    spec = df_this.iloc[0].spec

    for j in range(signals_f.shape[1]):
        ax.semilogy(range(signals_f.shape[0]), np.abs(signals_f[:, j, chosen_idx]), color=f"C{j}", label=f"mic{j}")

    yaw[np.isnan(yaw)] = 0
    axs[1].plot(range(signals_f.shape[0]), yaw)
    for deg in -np.arange(1, 5)*90:
        index = np.nanargmin(np.abs(yaw-deg))
        axs[1].axvline(x=index, color='C1')
        ax.axvline(x=index, color='C1')
        
    min_avg = 200
    max_avg = 300
    averages.append(np.sum(spec[min_avg:max_avg, chosen_idx], axis=0))
        
    ax.set_title(distance)
    ax.set_ylim(0.05, 10)
    ax.legend()
    [ax.grid() for ax in axs]

In [ ]:
plt.figure()
plt.scatter(distances, averages)
plt.xlabel('distance [cm]')
plt.title(f'average PSD at {freq} Hz')